# Wetlands LLM + MCP Testing

Test the LLM with MCP server integration using LangChain

In [17]:
import os
import json
import requests
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

## Configuration

In [18]:
# Load config
with open('config.json') as f:
    config = json.load(f)

# Get API key from environment
api_key = os.getenv('NRP_API_KEY')
if not api_key:
    print("WARNING: NRP_API_KEY not set!")
    api_key = input("Enter your NRP API key: ")

# MCP server URL
mcp_url = config['mcp_server_url']
llm_endpoint = config['llm_endpoint']
llm_model = config['llm_model']

print(f"LLM Endpoint: {llm_endpoint}")
print(f"Model: {llm_model}")
print(f"MCP Server: {mcp_url}")

LLM Endpoint: https://ellm.nrp-nautilus.io/v1
Model: glm-v
MCP Server: http://localhost:8001/mcp


## Load System Prompt

In [19]:
with open('system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt loaded: {len(system_prompt)} characters")

System prompt loaded: 5131 characters


## Define MCP Query Tool

In [20]:
def query_mcp(sql_query: str) -> str:
    """Execute SQL query via MCP server"""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": "query",
            "arguments": {
                "query": sql_query
            }
        }
    }
    
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    
    response = requests.post(mcp_url, json=payload, headers=headers)
    response.raise_for_status()
    
    data = response.json()
    if 'error' in data:
        raise Exception(f"MCP error: {data['error']['message']}")
    
    return data['result']['content'][0]['text']

@tool
def query_wetlands_data(query: str) -> str:
    """Execute a SQL query on the wetlands database using DuckDB.
    
    Args:
        query: SQL query to execute. Must start with S3 secret setup.
    
    Returns:
        Query results as a string
    """
    return query_mcp(query)

## Initialize LLM

In [21]:
# Initialize LangChain LLM
llm = ChatOpenAI(
    base_url=llm_endpoint,
    api_key=api_key,
    model=llm_model,
    temperature=0.7
)

# Bind the tool
llm_with_tools = llm.bind_tools([query_wetlands_data])

print("LLM initialized with tools")

LLM initialized with tools


## Test Direct MCP Query

In [22]:
# Test MCP server directly
test_query = """
CREATE OR REPLACE SECRET s3secret (
    TYPE S3,
    PROVIDER CONFIG,
    ENDPOINT 'minio.carlboettiger.info',
    USE_SSL true,
    URL_STYLE 'path'
);

SELECT COUNT(*) as total_hexagons 
FROM read_parquet('s3://public-wetlands/hex/**')
WHERE Z > 0;
"""

result = query_mcp(test_query)
print("MCP Test Result:")
print(result)

MCP Test Result:
+----------------+
| total_hexagons |
|     BIGINT     |
+----------------+
|   647463329    |
+----------------+


## Test LLM Without Tool Calling

In [23]:
# Simple test without tools
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is 2+2?")
]

response = llm.invoke(messages)
print("LLM Test (no tools):")
print(response.content)

LLM Test (no tools):

The sum of 2 and 2 is 4. 

**Answer:** 4


## Test LLM With Tool Calling

In [ ]:
def chat_with_tools(user_message: str):
    """Send message to LLM and handle tool calls"""
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message)
    ]
    
    # Get LLM response
    response = llm_with_tools.invoke(messages)
    
    print(f"\n{'='*60}")
    print(f"User: {user_message}")
    print(f"{'='*60}\n")
    
    # Check if LLM wants to call a tool
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"🔧 LLM is calling tool: {tool_call['name']}\n")
            
            # Extract SQL query from tool call args
            args = tool_call.get('args', {})
            
            # Check if args is empty or doesn't have 'query' key
            if not args or 'query' not in args:
                print(f"⚠️ Empty or invalid args from LLM: {args}")
                print("LLM failed to provide query. Asking for plain text response instead...\n")
                
                # Get a non-tool response instead
                plain_response = llm.invoke(messages)
                print("🤖 Assistant:")
                print(plain_response.content)
                return plain_response.content
            
            sql_query = args['query']
            
            print("SQL Query:")
            print("-" * 60)
            print(sql_query)
            print("-" * 60)
            
            try:
                tool_result = query_wetlands_data.invoke({'query': sql_query})
                print("\nQuery Result:")
                print("-" * 60)
                print(tool_result)
                print("-" * 60)
                
                # Send result back to LLM for interpretation
                messages.append(response)
                messages.append(
                    HumanMessage(
                        content=f"Tool result: {tool_result}",
                        name="tool_result"
                    )
                )
                
                final_response = llm.invoke(messages)
                print("\n🤖 Assistant:")
                print(final_response.content)
                return final_response.content
                
            except Exception as e:
                print(f"\n❌ Error executing query: {e}")
                print("Asking LLM to try again...\n")
                
                # Tell LLM about the error and ask it to try again
                messages.append(response)
                messages.append(
                    HumanMessage(
                        content=f"Error executing query: {e}\n\nPlease provide a corrected SQL query."
                    )
                )
                
                retry_response = llm.invoke(messages)
                print("🤖 Assistant (retry):")
                print(retry_response.content)
                return retry_response.content
    else:
        print("🤖 Assistant:")
        print(response.content)
        return response.content

## Example Queries

In [25]:
# Query 1: Total wetlands area
chat_with_tools("How many hectares of wetlands are there in total?")


User: How many hectares of wetlands are there in total?

🔧 LLM is calling tool: query_wetlands_data

DEBUG - tool_call structure: {'name': 'query_wetlands_data', 'args': {}, 'id': 'chatcmpl-tool-e9e8835e13f34c9dabf1273b8488fda4', 'type': 'tool_call'}
SQL Query:
------------------------------------------------------------
{}
------------------------------------------------------------

Query Result:
------------------------------------------------------------
Error executing tool query: ❌ Error executing query: Parser Error: syntax error at or near "{"

LINE 1: {}
        ^
------------------------------------------------------------

🤖 Assistant:

I apologize for the error. Let me try again with a properly formatted query to calculate the total wetland area:
<tool_call>query_wetlands_data
<|begin_of_box|>{"query": "CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');\n\nSELECT \n    COUNT(*) as total_hexagons,\n    ROUND(COUNT(*) * 73.7327598, 

'\nI apologize for the error. Let me try again with a properly formatted query to calculate the total wetland area:\n<tool_call>query_wetlands_data\n<|begin_of_box|>{"query": "CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT \'minio.carlboettiger.info\', URL_STYLE \'path\');\\n\\nSELECT \\n    COUNT(*) as total_hexagons,\\n    ROUND(COUNT(*) * 73.7327598, 2) as total_hectares,\\n    ROUND(COUNT(*) * 0.737327598, 2) as total_km2\\nFROM read_parquet(\'s3://public-wetlands/hex/**\')\\nWHERE Z > 0;"}<|end_of_box|>'

In [26]:
# Query 2: Peatlands
chat_with_tools("What is the total area of peatlands in square kilometers?")


User: What is the total area of peatlands in square kilometers?

🔧 LLM is calling tool: query_wetlands_data

DEBUG - tool_call structure: {'name': 'query_wetlands_data', 'args': {'query': "CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');\n\nSELECT \n    'Peatlands (codes 24-31)' as wetland_group,\n    COUNT(*) as total_hexagons,\n    ROUND(COUNT(*) * 0.737327598, 2) as total_km2\nFROM read_parquet('s3://public-wetlands/hex/**')\nWHERE Z BETWEEN 24 AND 31;"}, 'id': 'chatcmpl-tool-2096fb2231c54bd78b7f10ca82359955', 'type': 'tool_call'}
SQL Query:
------------------------------------------------------------
CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');

SELECT 
    'Peatlands (codes 24-31)' as wetland_group,
    COUNT(*) as total_hexagons,
    ROUND(COUNT(*) * 0.737327598, 2) as total_km2
FROM read_parquet('s3://public-wetlands/hex/**')
WHERE Z BETWEEN 24 AND 31;
---------------------------------

"\nBased on the analysis of the global wetlands database, the total area of peatlands worldwide is **100,672,986.36 square kilometers**.\n\nThis represents approximately 136.5 million H3 hexagons (each covering about 0.737 km²), which is a substantial portion of the Earth's wetland ecosystems. Peatlands are critical carbon sinks and biodiversity hotspots, storing vast amounts of carbon and supporting unique plant and animal communities.\n\nThis figure includes all peatland types (bogs, fens, mires, string bogs, palsa, peatland forests, tundra wetlands, and alpine wetlands) as classified in the global wetlands dataset.\n\nWould you like to explore specific regions or types of peatlands in more detail?"

In [27]:
# Query 3: Freshwater vs saline
chat_with_tools("Compare the area of freshwater wetlands to saline wetlands")


User: Compare the area of freshwater wetlands to saline wetlands

🔧 LLM is calling tool: query_wetlands_data

DEBUG - tool_call structure: {'name': 'query_wetlands_data', 'args': {}, 'id': 'chatcmpl-tool-8e182ada2a9b4f658d479575b667339e', 'type': 'tool_call'}
SQL Query:
------------------------------------------------------------
{}
------------------------------------------------------------

Query Result:
------------------------------------------------------------
Error executing tool query: ❌ Error executing query: Parser Error: syntax error at or near "{"

LINE 1: {}
        ^
------------------------------------------------------------

🤖 Assistant:

I apologize for the error. It looks like there was an issue with the query format. Let me try again with a properly formatted query to compare freshwater and saline wetlands.
<tool_call>query_wetlands_data
<tool_call>

🤖 Assistant:

I apologize for the error. It looks like there was an issue with the query format. Let me try again w

'\nI apologize for the error. It looks like there was an issue with the query format. Let me try again with a properly formatted query to compare freshwater and saline wetlands.\n<tool_call>query_wetlands_data\n<tool_call>'

In [ ]:
# Your custom query
user_query = "What are the top 5 most common wetland types by area?"
chat_with_tools(user_query)

## Troubleshooting

If the LLM returns empty args, the model may not support tool calling properly. You can:
1. Try a different model (e.g., `gpt-4` or `gpt-3.5-turbo`)
2. Manually construct queries and call `query_mcp()` directly
3. Use the MCP server test above to verify queries work